In [1]:
import glob
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix, plot_stat_map
import os
import pandas as pd
import pickle
import re
import copy

import sys
sys.path.append('/Users/zeynepenkavi/Documents/RangelLab/DescribedVsLearned_fmri/analysis/01_level1')
from level1_utils import run_level1, make_level1_design_matrix, get_events

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
data_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/'
behavior_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/behavioral_data/all_trials.csv'

subnum = '01'
runnum='4'

to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 0)
plot_design_matrix(design_matrix)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1a', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix = make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [7]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model4', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valChosen_par,valUnchosen_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,0.229434,-0.353206
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,0.178953,-0.277252
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,-0.324317,-0.077293
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,0.142307,-0.218429
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,0.161027,-0.330192
valChosen_par,0.229434,0.178953,-0.324317,0.142307,0.161027,1.000000,0.227738
valUnchosen_par,-0.353206,-0.277252,-0.077293,-0.218429,-0.330192,0.227738,1.000000


In [8]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valDiffFractal_par,valDiffLottery_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,0.066875,0.001632
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,0.048696,-0.001519
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,-0.055300,0.047054
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,0.034670,-0.001794
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,0.145922,0.009642
valDiffFractal_par,0.066875,0.048696,-0.055300,0.034670,0.145922,1.000000,-0.082635
valDiffLottery_par,0.001632,-0.001519,0.047054,-0.001794,0.009642,-0.082635,1.000000


In [9]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5a', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valDiffFractalWeighted_par,valDiffLotteryWeighted_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,0.008549,-0.005146
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,0.001036,-0.003685
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,0.010299,0.085413
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,-0.004034,-0.004963
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,0.139205,-0.005466
valDiffFractalWeighted_par,0.008549,0.001036,0.010299,-0.004034,0.139205,1.000000,0.007395
valDiffLotteryWeighted_par,-0.005146,-0.003685,0.085413,-0.004963,-0.005466,0.007395,1.000000


In [10]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model5b', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valDiffFractalHighPFrac_par,valDiffFractalLowPFrac_par,valDiffFractalMedPFrac_par,valDiffLotteryHighPFrac_par,valDiffLotteryLowPFrac_par,valDiffLotteryMedPFrac_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,0.042387,0.086212,-0.068118,-0.005873,-0.071553,0.052195
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,0.025391,0.069478,-0.050327,-0.009865,-0.054140,0.039609
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,0.033067,-0.114490,-0.024351,-0.033413,0.094793,0.029488
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,0.013470,0.054741,-0.035756,-0.003997,-0.043650,0.027343
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,0.213766,0.048159,-0.125460,-0.015391,-0.076721,0.091500
valDiffFractalHighPFrac_par,0.042387,0.025391,0.033067,0.013470,0.213766,1.000000,-0.018410,-0.009124,-0.096257,0.019759,0.010689
valDiffFractalLowPFrac_par,0.086212,0.069478,-0.114490,0.054741,0.048159,-0.018410,1.000000,-0.050995,-0.001998,-0.220486,-0.021388
valDiffFractalMedPFrac_par,-0.068118,-0.050327,-0.024351,-0.035756,-0.125460,-0.009124,-0.050995,1.000000,-0.000174,-0.021989,-0.079617
valDiffLotteryHighPFrac_par,-0.005873,-0.009865,-0.033413,-0.003997,-0.015391,-0.096257,-0.001998,-0.000174,1.000000,0.002453,-0.020237
valDiffLotteryLowPFrac_par,-0.071553,-0.054140,0.094793,-0.043650,-0.076721,0.019759,-0.220486,-0.021989,0.002453,1.000000,0.005228


In [11]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model6', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

A 'modulation' column was found in the given events data and is used.


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


,choiceShift_st,fractalProb_ev,fractalProb_par,reward_ev,stim_ev,valChosenFractal_par,valChosenLottery_par,valUnchosenFractal_par,valUnchosenLottery_par
choiceShift_st,1.000000,0.590639,0.024667,0.757393,0.837817,-0.023668,0.272240,-0.085956,-0.238110
fractalProb_ev,0.590639,1.000000,0.006043,0.027088,0.645227,-0.016337,0.209683,-0.065263,-0.187216
fractalProb_par,0.024667,0.006043,1.000000,0.032470,-0.098054,0.129578,-0.129961,-0.297794,0.179494
reward_ev,0.757393,0.027088,0.032470,1.000000,0.498400,-0.007046,0.162809,-0.048378,-0.146209
stim_ev,0.837817,0.645227,-0.098054,0.498400,1.000000,-0.123462,0.293492,-0.127539,-0.245849
valChosenFractal_par,-0.023668,-0.016337,0.129578,-0.007046,-0.123462,1.000000,0.135699,0.165298,0.226122
valChosenLottery_par,0.272240,0.209683,-0.129961,0.162809,0.293492,0.135699,1.000000,-0.114630,0.064760
valUnchosenFractal_par,-0.085956,-0.065263,-0.297794,-0.048378,-0.127539,0.165298,-0.114630,1.000000,-0.201076
valUnchosenLottery_par,-0.238110,-0.187216,0.179494,-0.146209,-0.245849,0.226122,0.064760,-0.201076,1.000000


In [ ]:
mnum = 'model1'
reg_rt = 1
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt%s")%(mnum, str(reg_rt)

# run_level1(subnum, mnum, data_path, behavior_path, out_path, regress_rt = reg_rt)

In [ ]:
mnum = 'model2'
reg_rt = 0
out_path = os.path.join(data_path, "derivatives/nilearn/glm/level1/%s_reg-rt%s")%(mnum, str(reg_rt))
                                                                                
# run_level1(subnum, 'model2', data_path, behavior_path, out_path, regress_rt = 0)